In [1]:
import numpy as np
import scipy
import h5py
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.optimizers import Adam,RMSprop 

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Flatten, Dense,Conv2D, MaxPooling2D,Dropout,BatchNormalization
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.applications import inception_v3

from PIL import Image

C:\Users\ali\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model = inception_v3.InceptionV3()

model_input_layer = model.layers[0].input  # First Layer
model_output_layer = model.layers[-1].output # Last Layer

# Choose an ImageNet object to fake
# Class #71 is "Scorpio"
object_type_to_fake = 71

# Load the image to hack
img = image.load_img("cat_original.jpg", target_size=(299, 299))
original_image = image.img_to_array(img)

# Scale the image so all pixel intensities are between [-1, 1] as the model expects
original_image /= 255.
original_image -= 0.5
original_image *= 2.

# Add a 4th dimension for batch size (as Keras expects)
original_image = np.expand_dims(original_image, axis=0)

# Pre-calculate the maximum change we will allow to the image
# We'll make sure our hacked image never goes past this so it doesn't look funny.
# A larger number produces an image faster but risks more distortion.
max_change_above = original_image + 0.01
max_change_below = original_image - 0.01

# Create a copy of the input image to hack on
hacked_image = np.copy(original_image)

In [3]:
# How much to update the hacked image in each iteration
learning_rate = 0.2

# Define the cost function.
# Our 'cost' will be the likelihood out image is the target class according to the pre-trained model
cost_function = model_output_layer[0, object_type_to_fake]

# We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
# In this case, referring to "model_input_layer" will give us back image we are hacking.
gradient_function = K.gradients(cost_function, model_input_layer)[0]

# Create a Keras function that we can call to calculate the current cost and gradient
grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

cost = 0.0

In [4]:

# In a loop, keep adjusting the hacked image slightly so that it tricks the model more and more
# until it gets to at least 90% confidence
while cost < 0.90:
    # Check how close the image is to our target class and grab the gradients we
    # can use to push it one more step in that direction.
    # Note: It's really important to pass in '0' for the Keras learning mode here!
    # Keras layers behave differently in prediction vs. train modes!
    cost, gradients = grab_cost_and_gradients_from_model([hacked_image, 0])

    # Move the hacked image one step further towards fooling the model
    hacked_image += gradients * learning_rate

    # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
    hacked_image = np.clip(hacked_image, max_change_below, max_change_above)
    hacked_image = np.clip(hacked_image, -1.0, 1.0)

    print("Model's predicted likelihood that the image is a scorpion {:.8}%".format(cost * 100))



Model's predicted likelihood that the image is a scorpion 0.0069180518%
Model's predicted likelihood that the image is a scorpion 0.0069224363%
Model's predicted likelihood that the image is a scorpion 0.0069267939%
Model's predicted likelihood that the image is a scorpion 0.0069311754%
Model's predicted likelihood that the image is a scorpion 0.006935509%
Model's predicted likelihood that the image is a scorpion 0.0069398782%
Model's predicted likelihood that the image is a scorpion 0.0069442351%
Model's predicted likelihood that the image is a scorpion 0.0069486625%
Model's predicted likelihood that the image is a scorpion 0.0069530703%
Model's predicted likelihood that the image is a scorpion 0.0069574562%
Model's predicted likelihood that the image is a scorpion 0.0069618269%
Model's predicted likelihood that the image is a scorpion 0.0069662165%
Model's predicted likelihood that the image is a scorpion 0.0069706402%
Model's predicted likelihood that the image is a scorpion 0.00697

Model's predicted likelihood that the image is a scorpion 0.0074369687%
Model's predicted likelihood that the image is a scorpion 0.007441653%
Model's predicted likelihood that the image is a scorpion 0.0074463525%
Model's predicted likelihood that the image is a scorpion 0.0074510172%
Model's predicted likelihood that the image is a scorpion 0.0074557065%
Model's predicted likelihood that the image is a scorpion 0.0074603973%
Model's predicted likelihood that the image is a scorpion 0.0074651136%
Model's predicted likelihood that the image is a scorpion 0.0074698233%
Model's predicted likelihood that the image is a scorpion 0.0074745461%
Model's predicted likelihood that the image is a scorpion 0.007479285%
Model's predicted likelihood that the image is a scorpion 0.0074840151%
Model's predicted likelihood that the image is a scorpion 0.0074887459%
Model's predicted likelihood that the image is a scorpion 0.0074934891%
Model's predicted likelihood that the image is a scorpion 0.007498

Model's predicted likelihood that the image is a scorpion 0.0080092446%
Model's predicted likelihood that the image is a scorpion 0.0080143654%
Model's predicted likelihood that the image is a scorpion 0.0080194593%
Model's predicted likelihood that the image is a scorpion 0.008024554%
Model's predicted likelihood that the image is a scorpion 0.0080296471%
Model's predicted likelihood that the image is a scorpion 0.0080347425%
Model's predicted likelihood that the image is a scorpion 0.0080398582%
Model's predicted likelihood that the image is a scorpion 0.0080449805%
Model's predicted likelihood that the image is a scorpion 0.0080501297%
Model's predicted likelihood that the image is a scorpion 0.008055276%
Model's predicted likelihood that the image is a scorpion 0.0080604448%
Model's predicted likelihood that the image is a scorpion 0.008065618%
Model's predicted likelihood that the image is a scorpion 0.008070816%
Model's predicted likelihood that the image is a scorpion 0.00807604

Model's predicted likelihood that the image is a scorpion 0.0086337757%
Model's predicted likelihood that the image is a scorpion 0.0086395594%
Model's predicted likelihood that the image is a scorpion 0.008645327%
Model's predicted likelihood that the image is a scorpion 0.0086510605%
Model's predicted likelihood that the image is a scorpion 0.0086568136%
Model's predicted likelihood that the image is a scorpion 0.0086625601%
Model's predicted likelihood that the image is a scorpion 0.0086682914%
Model's predicted likelihood that the image is a scorpion 0.0086740576%
Model's predicted likelihood that the image is a scorpion 0.0086798347%
Model's predicted likelihood that the image is a scorpion 0.0086856089%
Model's predicted likelihood that the image is a scorpion 0.0086914028%
Model's predicted likelihood that the image is a scorpion 0.0086972577%
Model's predicted likelihood that the image is a scorpion 0.0087031127%
Model's predicted likelihood that the image is a scorpion 0.00870

Model's predicted likelihood that the image is a scorpion 0.009334393%
Model's predicted likelihood that the image is a scorpion 0.0093408409%
Model's predicted likelihood that the image is a scorpion 0.0093473405%
Model's predicted likelihood that the image is a scorpion 0.0093538089%
Model's predicted likelihood that the image is a scorpion 0.0093603274%
Model's predicted likelihood that the image is a scorpion 0.0093668721%
Model's predicted likelihood that the image is a scorpion 0.0093733936%
Model's predicted likelihood that the image is a scorpion 0.0093799375%
Model's predicted likelihood that the image is a scorpion 0.0093865034%
Model's predicted likelihood that the image is a scorpion 0.009393059%
Model's predicted likelihood that the image is a scorpion 0.0093995877%
Model's predicted likelihood that the image is a scorpion 0.0094061637%
Model's predicted likelihood that the image is a scorpion 0.009412763%
Model's predicted likelihood that the image is a scorpion 0.0094193

...

Model's predicted likelihood that the image is a scorpion 0.071501679%
Model's predicted likelihood that the image is a scorpion 0.071645627%
Model's predicted likelihood that the image is a scorpion 0.071790552%
Model's predicted likelihood that the image is a scorpion 0.071934244%
Model's predicted likelihood that the image is a scorpion 0.072076824%
Model's predicted likelihood that the image is a scorpion 0.072219456%
Model's predicted likelihood that the image is a scorpion 0.072361081%
Model's predicted likelihood that the image is a scorpion 0.072502467%
Model's predicted likelihood that the image is a scorpion 0.072644628%
Model's predicted likelihood that the image is a scorpion 0.072788936%
Model's predicted likelihood that the image is a scorpion 0.072933035%
Model's predicted likelihood that the image is a scorpion 0.073077832%
Model's predicted likelihood that the image is a scorpion 0.073223026%
Model's predicted likelihood that the image is a scorpion 0.07336812%
Model's

Model's predicted likelihood that the image is a scorpion 0.091772579%
Model's predicted likelihood that the image is a scorpion 0.091976777%
Model's predicted likelihood that the image is a scorpion 0.092184835%
Model's predicted likelihood that the image is a scorpion 0.092396111%
Model's predicted likelihood that the image is a scorpion 0.092611235%
Model's predicted likelihood that the image is a scorpion 0.09283612%
Model's predicted likelihood that the image is a scorpion 0.09306391%
Model's predicted likelihood that the image is a scorpion 0.093295623%
Model's predicted likelihood that the image is a scorpion 0.093529944%
Model's predicted likelihood that the image is a scorpion 0.093760603%
Model's predicted likelihood that the image is a scorpion 0.09399394%
Model's predicted likelihood that the image is a scorpion 0.094227184%
Model's predicted likelihood that the image is a scorpion 0.094463525%
Model's predicted likelihood that the image is a scorpion 0.094703498%
Model's p

Model's predicted likelihood that the image is a scorpion 0.12778307%
Model's predicted likelihood that the image is a scorpion 0.12822653%
Model's predicted likelihood that the image is a scorpion 0.12868089%
Model's predicted likelihood that the image is a scorpion 0.12913854%
Model's predicted likelihood that the image is a scorpion 0.12961555%
Model's predicted likelihood that the image is a scorpion 0.13009292%
Model's predicted likelihood that the image is a scorpion 0.13057585%
Model's predicted likelihood that the image is a scorpion 0.13106366%
Model's predicted likelihood that the image is a scorpion 0.1315508%
Model's predicted likelihood that the image is a scorpion 0.13204501%
Model's predicted likelihood that the image is a scorpion 0.13255692%
Model's predicted likelihood that the image is a scorpion 0.13307165%
Model's predicted likelihood that the image is a scorpion 0.13362824%
Model's predicted likelihood that the image is a scorpion 0.13417701%
Model's predicted lik

Model's predicted likelihood that the image is a scorpion 0.21735523%
Model's predicted likelihood that the image is a scorpion 0.2184839%
Model's predicted likelihood that the image is a scorpion 0.21961725%
Model's predicted likelihood that the image is a scorpion 0.22078317%
Model's predicted likelihood that the image is a scorpion 0.22194933%
Model's predicted likelihood that the image is a scorpion 0.22314282%
Model's predicted likelihood that the image is a scorpion 0.22434318%
Model's predicted likelihood that the image is a scorpion 0.22558225%
Model's predicted likelihood that the image is a scorpion 0.22684659%
Model's predicted likelihood that the image is a scorpion 0.22813443%
Model's predicted likelihood that the image is a scorpion 0.22942803%
Model's predicted likelihood that the image is a scorpion 0.23073673%
Model's predicted likelihood that the image is a scorpion 0.2320562%
Model's predicted likelihood that the image is a scorpion 0.23340092%
Model's predicted like

Model's predicted likelihood that the image is a scorpion 0.52495403%
Model's predicted likelihood that the image is a scorpion 0.53072232%
Model's predicted likelihood that the image is a scorpion 0.53671841%
Model's predicted likelihood that the image is a scorpion 0.54319985%
Model's predicted likelihood that the image is a scorpion 0.54942947%
Model's predicted likelihood that the image is a scorpion 0.55614775%
Model's predicted likelihood that the image is a scorpion 0.56268307%
Model's predicted likelihood that the image is a scorpion 0.5692625%
Model's predicted likelihood that the image is a scorpion 0.57632625%
Model's predicted likelihood that the image is a scorpion 0.58385143%
Model's predicted likelihood that the image is a scorpion 0.59135687%
Model's predicted likelihood that the image is a scorpion 0.59912242%
Model's predicted likelihood that the image is a scorpion 0.60711107%
Model's predicted likelihood that the image is a scorpion 0.61520571%
Model's predicted lik

In [6]:
# De-scale the image's pixels from [-1, 1] back to the [0, 255] range
img = hacked_image[0]
img /= 2.
img += 0.5
img *= 255.

# Save the hacked image!
im = Image.fromarray(img.astype(np.uint8))
im.save("hacked-image.png")